In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kg20c-scholarly-knowledge-graph/valid.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/all_relation_info.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/all_entity_info.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/README.md
/kaggle/input/kg20c-scholarly-knowledge-graph/test.txt
/kaggle/input/kg20c-scholarly-knowledge-graph/train.txt


In [5]:
ROOT = '/kaggle/input/kg20c-scholarly-knowledge-graph/'

def get_txt_pd(columns, file_name):
    main = pd.read_csv(ROOT+file_name, sep='\t', header=None)
    main.columns = columns
    main = main.iloc[1:]
    return main

entity_ref = get_txt_pd(['id', 'name', 'type'], 'all_entity_info.txt')
train_data = get_txt_pd(['paper1', 'relation', 'paper2'], 'train.txt')

with open(ROOT+'all_relation_info.txt', 'r') as rel:
    data = rel.readlines()
    rem_new = lambda x: x.replace('\n', '')
    refs = [rem_new(x) for x in data]

vals = [i for i in range(len(refs))]
relation_ref = dict(zip(refs, vals))

relation_ref['self'] = 6

print(relation_ref)
all_nodes = list(set(entity_ref['id']))

np_table = np.identity(len(all_nodes)) * relation_ref['self']

{'id': 0, 'author_in_affiliation': 1, 'author_write_paper': 2, 'paper_cite_paper': 3, 'paper_in_domain': 4, 'paper_in_venue': 5, 'self': 6}


In [6]:
adjacency = pd.DataFrame(np_table, columns=all_nodes, index=all_nodes)

train_data['relation'] = train_data['relation'].replace(relation_ref)
train_list = train_data.values.tolist()
print(train_list[0])

for relation in train_list:
    adjacency[relation[0]][relation[2]] = relation[1]


['062B7298', 1, '00D4F325']


In [9]:
n_table = adjacency.to_numpy()
print(np.count_nonzero(n_table == 3))

adjacency.head()

7382


,7E8EB778,8005485D,782E7DE2,83AB29DD,139EEE26,8166DDF3,80E5A7AB,7F641EC4,54065E3E,0FC839F7,...,0650B2E2,5A4BA738,5281ABB3,81482730,01A73FE2,7ED5FC36,04EDE9AE,58C5DD66,861F4100,7B4246D6
7E8EB778,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8005485D,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
782E7DE2,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83AB29DD,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139EEE26,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
!pip3 install paperscraper

In [13]:
# from paperscraper import PaperScraper
# scraper = PaperScraper()

# !pip install google-search-results

from bs4 import BeautifulSoup
import requests
import json
import bs4 as bs 
import urllib.request 
from urllib.request import Request, urlopen 
import re
from serpapi import GoogleSearch


all_titles = entity_ref[['id', 'name']]
all_titles['url'] = ''

proxies = {
"http": 'http://209.50.52.162:9050', 
"https": 'http://209.50.52.162:9050'
}

all_names = all_titles['name'].tolist()
query = '+'.join(all_names[0].split(' '))
print(query)

url_template = f"https://serpapi.com/search.json?engine=google_scholar&q={query}"
print(url_template)


params = {
  "engine": "google_scholar",
  "q": query,
  "hl": "en",
  "api_key": "c9fd1f9e1aabf01316abde7496afa68c38493d0a9bd94b7739fa52e0b837e460"
}

search = GoogleSearch(params)
results = search.get_dict()

On+rank+correlation+in+information+retrieval+evaluation
https://serpapi.com/search.json?engine=google_scholar&q=On+rank+correlation+in+information+retrieval+evaluation
https://serpapi.com/search
{'search_metadata': {'id': '63260981aa6c95af692092fe', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/cd33be5c2f654ed2/63260981aa6c95af692092fe.json', 'created_at': '2022-09-17 17:53:05 UTC', 'processed_at': '2022-09-17 17:53:05 UTC', 'google_scholar_url': 'https://scholar.google.com/scholar?q=On%2Brank%2Bcorrelation%2Bin%2Binformation%2Bretrieval%2Bevaluation&hl=en', 'raw_html_file': 'https://serpapi.com/searches/cd33be5c2f654ed2/63260981aa6c95af692092fe.html', 'total_time_taken': 0.45}, 'search_parameters': {'engine': 'google_scholar', 'q': 'On+rank+correlation+in+information+retrieval+evaluation', 'hl': 'en'}, 'search_information': {'organic_results_state': 'Results for exact spelling', 'query_displayed': 'On+rank+correlation+in+information+retrieval+evaluation'}, 'profi

In [21]:
scrape_link = results['organic_results'][0]['link']
print(scrape_link)

https://dl.acm.org/doi/abs/10.1145/1273221.1273223


In [35]:
import requests
from bs4 import BeautifulSoup
page = requests.get(scrape_link)
soup = BeautifulSoup(page.content, "html.parser")
print(soup)

<!DOCTYPE html>

<html class="pb-page" data-request-id="64f0dcd4-9cc9-4f33-a9bd-9fd16e90a873" lang="en"><head data-pb-dropzone="head"><meta content=";csubtype:string:Newsletter;wgroup:string:ACM Publication Websites;topic:topic:acmsig&gt;sigir;journal:journal:sigir;requestedJournal:journal:sigir;serialTopic:topic:acm-pubtype&gt;newsletter;page:string:Article/Chapter View;ctype:string:Journal Content;subPage:string:Abstract;article:article:doi\:10.1145/1273221.1273223;groupTopic:topic:acm-pubtype&gt;newsletter;website:website:dl-site;issue:issue:doi\:10.1145/1273221;taxonomy:taxonomy:acm-pubtype;pageGroup:string:Publication Pages" name="pbContext"/>
<link href="http://purl.org/DC/elements/1.0/" rel="schema.DC"/><meta content="ACM SIGIR Forum" name="citation_journal_title"/><meta content="On rank correlation in information retrieval evaluation" name="dc.Title"/><meta content="MelucciMassimo " name="dc.Creator"/><meta content="Some methods for rank correlation in evaluation are examined a

In [60]:
results = soup.find('div', class_='abstractSection abstractInFull').text
print(results)

<div class="abstractSection abstractInFull"><div class="abstractSection abstractInFull"><p>Some methods for rank correlation in evaluation are examined and their relative advantages and disadvantages are discussed. In particular, it is suggested that different test statistics should be used for providing additional information about the experiments other that the one provided by statistical significance testing. Kendall's τ is often used for testing-rank correlation, yet it is little appropriate if the objective of the test is different from what τ was designed for. In particular, attention should be paid to the null hypothesis. Other measures for rank correlation are described. If one test statistic suggests to reject a hypothesis, other test statistics should be used to support or to revise the decision. The paper then focuses on rank correlation between webpage lists ordered by PageRank for applying the general reflections on these test statistics. An interpretation of PageRank beha